#Perform date and time manipulation

** Data Source **
* English Wikipedia pageviews by second
* Size on Disk: ~255 MB
* Type: Parquet files
* More Info: <a href="https://datahub.io/en/dataset/english-wikipedia-pageviews-by-second" target="_blank">https&#58;//datahub.io/en/dataset/english-wikipedia-pageviews-by-second</a>

**Technical Accomplishments:**
* Explore more of the `...sql.functions` operations
  * Date & time functions

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [3]:
%run "./Includes/Classroom-Setup"

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) The Data Source

This data uses the **Pageviews By Seconds** data set.

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# I've already gone through the exercise to determine
# how many partitions I want and in this case it is...
partitions = 8

# Make sure wide operations don't repartition to 200
spark.conf.set("spark.sql.shuffle.partitions", str(partitions))

In [6]:
(source, sasEntity, sasToken) = getAzureDataSource()
spark.conf.set(sasEntity, sasToken)

# The directory containing our parquet files.
parquetFile = source + "/wikipedia/pageviews/pageviews_by_second.parquet/"

In [7]:
# Create our initial DataFrame. We can let it infer the 
# schema because the cost for parquet files is really low.
initialDF = (spark.read
  .option("inferSchema", "true") # The default, but not costly w/Parquet
  .parquet(parquetFile)          # Read the data in
  .repartition(partitions)       # From 7 >>> 8 partitions
  .cache()                       # Cache the expensive operation
)
# materialize the cache
initialDF.count()

-sandbox
##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Preparing Our Data

If we will be working on any given dataset for a while, there are a handful of "necessary" steps to get us ready...

Most of which we've just knocked out above.

**Basic Steps**
0. <div style="text-decoration:line-through">Read the data in</div>
0. <div style="text-decoration:line-through">Balance the number of partitions to the number of slots</div>
0. <div style="text-decoration:line-through">Cache the data</div>
0. <div style="text-decoration:line-through">Adjust the `spark.sql.shuffle.partitions`</div>
0. Perform some basic ETL (i.e., convert strings to timestamp)
0. Possibly re-cache the data if the ETL was costly

What we haven't done is some of the basic ETL necessary to explore our data.

Namely, the problem is that the field "timestamp" is a string.

In order to performed date/time - based computation I need to convert this to an alternate datetime format.

In [9]:
initialDF.printSchema()

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) withColumnRenamed(..), withColumn(..), select(..)

My first hangup is that we have a **column named timestamp** and the **datatype will also be timestamp**

The nice thing about Apache Spark is that I'm allowed the have an issue with this because it's very easy to fix...

Just rename the column...

In [11]:
(initialDF
  .select( col("timestamp").alias("capturedAt"), col("site"), col("requests") )
  .printSchema()
)

There are a number of different ways to rename a column...

In [13]:
(initialDF
  .withColumnRenamed("timestamp", "capturedAt")
  .printSchema()
)

In [14]:
(initialDF
  .toDF("capturedAt", "site", "requests")
  .printSchema()
)

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) unix_timestamp(..) & cast(..)

Now that **we** are over **my** hangup, we can focus on converting the **string** to a **timestamp**.

For this we will be looking at more of the functions in the `functions` package
* `pyspark.sql.functions` in the case of Python
* `org.apache.spark.sql.functions` in the case of Scala & Java

And so that we can watch the transformation, will will take one step at a time...

The first function is `unix_timestamp(..)`

If you look at the API docs, `unix_timestamp(..)` is described like this:
> Convert time string with given pattern (see <a href="http://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html" target="_blank">SimpleDateFormat</a>) to Unix time stamp (in seconds), return null if fail.

`SimpleDataFormat` is part of the Java API and provides support for parsing and formatting date and time values.

In order to know what format the data is in, let's take a look at the first row...

Comparing that value with the patterns express in the docs for the `SimpleDateFormat` class, we can come up with a format:

**yyyy-MM-dd HH:mm:ss**

In [19]:
tempA = (initialDF
  .withColumnRenamed("timestamp", "capturedAt")
  .select( col("*"), unix_timestamp( col("capturedAt"), "yyyy-MM-dd HH:mm:ss") )
)
tempA.printSchema()

In [20]:
display(tempA)

** *Note:* ** *If you haven't caught it yet, there is a bug in the previous code....*

A couple of things happened...
0. We ended up with a new column - that's OK for now
0. The new column has a really funky name - based upon the name of the function we called and its parameters.
0. The data type is now a long.
  * This value is the Java Epoch
  * The number of seconds since 1970-01-01T00:00:00Z
  
We can now take that epoch value and use the `Column.cast(..)` method to convert it to a **timestamp**.

In [23]:
tempB = (initialDF
  .withColumnRenamed("timestamp", "capturedAt")
  .select( col("*"), unix_timestamp( col("capturedAt"), "yyyy-MM-dd'T'HH:mm:ss").cast("timestamp") )
)
tempB.printSchema()

In [24]:
display(tempB)

Now that our column `createdAt` has been converted from a **string** to a **timestamp**, we just need to deal with this REALLY funky column name.

Again.. there are several ways to do this.

I'll let you decide which you like better...

### Option #1
The `as()` or `alias()` method can be appended to the chain of calls.

This version will actually produce an odd little bug.<br/>
That is, how do you get rid of only one of the two `capturedAt` columns?

In [27]:
tempC = (initialDF
  .withColumnRenamed("timestamp", "capturedAt")
  .select( col("*"), unix_timestamp( col("capturedAt"), "yyyy-MM-dd'T'HH:mm:ss").cast("timestamp").alias("capturedAt") )
)
tempC.printSchema()

In [28]:
display(tempC)

### Option #2
The `withColumn(..)` renames the column (first param) and accepts as a<br/>
second parameter the expression(s) we need for our transformation

In [30]:
tempD = (initialDF
  .withColumnRenamed("timestamp", "capturedAt")
  .withColumn("capturedAt", unix_timestamp( col("capturedAt"), "yyyy-MM-dd'T'HH:mm:ss").cast("timestamp") )
)
tempD.printSchema()

In [31]:
display(tempD)

### Option #3

We can take the big ugly name explicitly rename it.

This version will actually produce an odd little bug.<br/>
That is how do you get rid of only one of the two "capturedAt" columns?

In [33]:
#Option #3

tempE = (initialDF
  .withColumnRenamed("timestamp", "capturedAt")
  .select( col("*"), unix_timestamp( col("capturedAt"), "yyyy-MM-dd'T'HH:mm:ss").cast("timestamp") )
  .withColumnRenamed("CAST(unix_timestamp(capturedAt, yyyy-MM-dd'T'HH:mm:ss) AS TIMESTAMP)", "capturedAt")
  # .drop("timestamp")
)
tempE.printSchema()

In [34]:
display(tempE)

### Option #4

The last version is a twist on the others in which we start with the <br/>
name `timestamp` and rename it and the expression all in one call<br/>

But this version leaves us with the old column in the DF

In [36]:
tempF = (initialDF
  .withColumn("capturedAt", unix_timestamp( col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss").cast("timestamp") )
)
tempF.printSchema()

In [37]:
display(tempF)

Let's pick the "cleanest" version...

And with our base `DataFrame` in place we can start exploring the data a little...

In [39]:
pageviewsDF = (initialDF
  .withColumnRenamed("timestamp", "capturedAt")
  .withColumn("capturedAt", unix_timestamp( col("capturedAt"), "yyyy-MM-dd'T'HH:mm:ss").cast("timestamp") )
)

pageviewsDF.printSchema()

In [40]:
display(pageviewsDF)

And just so that we don't have to keep performing these transformations.... 

Mark the `DataFrame` as cached and then materialize the result.

In [42]:
pageviewsDF.cache().count()

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) year(..), month(..), dayofyear(..)

Let's take a look at some of the other date & time functions...

With that we can answer a simple question: When was this data captured.

We can start specifically with the year...

In [44]:
display(
  pageviewsDF
    .select( year( col("capturedAt")) ) # Every record converted to a single column - the year captured
    .distinct()                         # Reduce all years to the list of distinct years
)

Now let's take a look at in which months was this data captured...

In [46]:
display(
  pageviewsDF
    .select( month( col("capturedAt")) ) # Every record converted to a single column - the month captured
    .distinct()                          # Reduce all months to the list of distinct years
)

And of course this both can be combined as a single call...

In [48]:
(pageviewsDF
  .select( month(col("capturedAt")).alias("month"), year(col("capturedAt")).alias("year"))
  .distinct()
  .show()                     
)

It's pretty easy to see that the data was captured during March & April of 2015.

We will have more opportunities to play with the various date and time functions in the next exercise.

For now, let's just make sure to review them in the Spark API

## Next steps

Start the next lesson, [Use aggregate functions]($./2.Use-Aggregate-Functions)